# Beet segmentation model training

Date: 18.02.2024  
Authors: Gustav Schimmer & Philipp Friedrich  

**This notebook is purposed for training a YOLOv6 algorithm in detecting sugar beet plants on images.**  
  
  
Major steps are:
- Initialize YOLOv6 algorithm
- Train algorithm
- Validation of the model
- Inferencing YOLOv6 model on test data

This notebook is designed to run from [Google Colab](https://colab.research.google.com/?hl=de). Before we start we need to make sure to have a proper dataset we can use. In case you don't have a dataset yet and want to use own custom data you can follow the steps in this [Jupyter Notebook](beet_segmentation_data.ipynb). Otherwise you can use the dataset we provided in the YOLOv6 section of this project (custom_dataset). Your dataset should be stored in your Google Drive for accessing it via Google Colab.

## Import necessary libraries

In [1]:
import os
from google.colab import drive

ModuleNotFoundError: No module named 'google'

Make sure your your notebook uses GPU to speed up model training.

In [ ]:
!nvidia-smi

## Mount Drive for working in Google Colab

In [ ]:
drive.mount('/content/drive', force_remount=True)

In [ ]:
HOME = os.getcwd()
print(HOME)

## Initializing YOLOv6 algorithm

In [ ]:
# Download MT-YOLOv6 repository and install requirements
!git clone https://github.com/meituan/YOLOv6
%cd YOLOv6

In [ ]:
!pip install -r requirements.txt

In [ ]:
pwd

## Update dataset.yml

Before we train our model we need to update the data/dataset.yml file with the following code, letting the model know where the data is located.



```
# Please insure that your custom_dataset are put in same parent dir with YOLOv6_DIR
train: ./custom_dataset/images/train # train images
val: ./custom_dataset/images/val # val images

# whether it is coco dataset, only coco dataset should be set to True.
is_coco: False
# Classes
nc: 1  # number of classes
names: ['sugar beet']  # class names
```



## Copy dataset to YOLOv6 directory

In [ ]:
%cp -r /content/drive/MyDrive/custom_dataset /content/YOLOv6/custom_dataset

## Train model

Now we can train the model. Batch size and epochs can be adjusted. Try different settings to see which yield the best results.  

Note the structure of the basic training command: we call a specific script (tools/train.py), set a batch size argument (--batch 256), set a specific configuration file (--conf configs/yolov6s.py), pass our data.yaml file, and set what type of CUDA device we're using (--device 0).

The configuration file for training YOLOv6 comes with support for finetuning (e.g. calling configs/yolov6s_finetune.py vs starting from scratch (yolov6s.py). Finetuning will train faster though may not be as effective on unique datasets. It may make most sense if you're resuming training from a substantially similar domain, like adding more data to a previously completed training job.

Moreover, YOLOv6 supports a high number of arguments by default. Here's what all the options available to us are, their default value, and what they mean:


*  --data-path, default='./data/coco.yaml', type=str, help='path of dataset')
*   --conf-file, default='./configs/yolov6s.py', type=str, help='experiments description file')
*   --img-size, type=int, default=640, help='train, val image size (pixels)')
*  --batch-size, default=32, type=int, help='total batch size for all GPUs')
*   --epochs, default=400, type=int, help='number of total epochs to run')
*   --workers, default=8, type=int, help='number of data loading workers (default: 8)')
*   --device, default='0', type=str, help='cuda device, i.e. 0 or 0,1,2,3 or cpu')
*   --eval-interval, type=int, default=20, help='evaluate at every interval epochs')
*   --eval-final-only, action='store_true', help='only evaluate at the final epoch')
*   --heavy-eval-range, default=50,help='evaluating every epoch for last such epochs (can be jointly used with --eval-interval)')
*  --check-images, action='store_true', help='check images when initializing datasets')
*   --check-labels, action='store_true', help='check label files when initializing datasets')
*   --output-dir, default='./runs/train', type=str, help='path to save outputs')
*   --name, default='exp', type=str, help='experiment name, saved to output_dir/name')
*   --dist_url, type=str, default="default url: tcp://127.0.0.1:8888")
*   --gpu_count, type=int, default=0)
*   --local_rank, type=int, default=-1, help='DDP parameter')
*   --resume, type=str, default=None, help='resume the corresponding ckpt')

In [ ]:
%cd /content/YOLOv6
!python tools/train.py --batch 16 --conf configs/yolov6s_finetune.py --data data/dataset.yaml --fuse_ab --epochs 10 --img-size 512 --device 0

## Evaulate YOLOv6 Model Performance

YOLOv6 comes with a tools directory, one of which is for evaluation of model training. Evaluating the model's performance includes assessing the model's mean average precision (mAP), precision, and recall. For the uninitiated, a higher mAP score indicates our model is drawing the correct boxes in the right places.

Similar to the training argument, we do have parameters we can adjust for how we evaluate the model (like specifying confidence thresholds or modifying where the results are saved).


* --data, type=str, default='./data/coco.yaml', help='dataset.yaml path')
* --weights, type=str, default='./weights/yolov6s.pt', help='model.pt path(s)')
* --batch-size, type=int, default=32, help='batch size')
* --img-size, type=int, default=640, help='inference size (pixels)')
* --conf-thres, type=float, default=0.001, help='confidence threshold')
* --iou-thres, type=float, default=0.65, help='NMS IoU threshold')
* --task, default='val', help='val, or speed')
* --device, default='0', help='cuda device, i.e. 0 or 0,1,2,3 or cpu')
* --half, default=False, action='store_true', help='whether to use fp16 infer')
* --save_dir, type=str, default='runs/val/', help='evaluation save dir')
* --name, type=str, default='exp', help='save evaluation results to save_dir/name')

The evaluation command we'll run is:

In [ ]:
%cd /content/YOLOv6
#make sure to enter the correct path to the best_ckpt.pt
!python tools/eval.py --data data/dataset.yaml  --weights runs/train/exp/weights/best_ckpt.pt --img-size 512 --do_pr_metric True --plot_confusion_matrix --task val --device 0

# Inference Yolov6 on Test dataset

Similar to YOLOv6 evaluation, there's an inference script that comes with the repository's tools. We had tested this our before. This inference script allows us to pass a batch of images in a given directory on which the model will run its predictions.

In addition, this inference utility also comes with a number of arguments we can pass for things like displaying the labels on the predicted images (which is on by default), modifying the confidence level and NMS thresholds, and setting max detections.

* --weights, type=str, default='weights/yolov6s.pt', help='model path(s) for inference.')
* --source, type=str, default='data/images', help='the source path, e.g. image-file/dir.')
* --yaml, type=str, default='data/coco.yaml', help='data yaml file.')
* --img-size, type=int, default=640, help='the image-size(h,w) in inference size.')
* --conf-thres, type=float, default=0.25, help='confidence threshold for inference.')
* --iou-thres, type=float, default=0.45, help='NMS IoU threshold for inference.')
* --max-det, type=int, default=1000, help='maximal inferences per image.')
* --device, default='0', help='device to run our model i.e. 0 or 0,1,2,3 or cpu.')
* --save-txt, action='store_true', help='save results to *.txt.')
* --save-img, action='store_false', help='save visuallized inference results.')
* --classes, nargs='+', type=int, help='filter by classes, e.g. --classes 0, or --classes 0 2 3.')
* --agnostic-nms, action='store_true', help='class-agnostic NMS.')
* --project, default='runs/inference', help='save inference results to project/name.')
* --name, default='exp', help='save inference results to project/name.')
* --hide-labels, default=False, action='store_true', help='hide labels.')
* --hide-conf, default=False, action='store_true', help='hide confidences.')
* --half, action='store_true', help='whether to use FP16 half-precision inference.')

We'll run this command:

In [ ]:
%cd /content/YOLOv6
!python tools/infer.py --yaml data/dataset.yaml --weights runs/train/exp/weights/best_ckpt.pt --source custom_dataset/images/val --img-size 512 512  --project 'runs/inference/sugarbeets' --device 0

## Converting Yolov6 to ONNX

One additional utility YOLOv6 comes with is the ability to be converted to ONNX, a common model serialization format for easier portability across devices.

Converting to ONNX is a simple command, where we've correctly references our model's weights file:

In [ ]:
pip install onnx>=1.10.0

In [ ]:
!python ./deploy/ONNX/export_onnx.py \
    --weights ./runs/train/exp/weights/best_ckpt.pt \
    --img-size 512 512 \
    --simplify \
    --iou-thres 0.35 \
    --conf-thres 0.35 \
    --dynamic-batch \
    --ort \